In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
athletes = pd.read_csv('data/summerOly_athletes.csv')
hosts = pd.read_csv('data/summerOly_hosts.csv')
medal_counts = pd.read_csv('data/summerOly_medal_counts.csv')
programs = pd.read_csv('data/summerOly_programs.csv', encoding='utf-8')

In [15]:
medal_counts

,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896
...,...,...,...,...,...,...,...
1430,84,Qatar,0,0,1,1,2024
1431,84,Refugee Olympic Team,0,0,1,1,2024
1432,84,Singapore,0,0,1,1,2024
1433,84,Slovakia,0,0,1,1,2024
